In [ ]:
"""
Use a powerful, model to act as your expert prompt engineering partner. 
Its task is not to solve the final problem, but to generate a high-quality, 
optimized prompt that you can then use for your actual task.
"""

In [11]:
import litellm
from IPython.display import display, Markdown
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

MODEL_NAME = "openai/gpt-5"
MAX_TOKENS_DEFAULT = 5001

def get_completion(
    prompt,
    model=MODEL_NAME,
    max_tokens=MAX_TOKENS_DEFAULT,
    **kwargs
):
    if "gpt-5" in model:
        kwargs["max_completion_tokens"] = max_tokens
    else:
        kwargs["max_tokens"] = max_tokens
        
    parsed_messages = []

    if type(prompt) is str:
        parsed_messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
    else:
        parsed_messages = prompt

    response = litellm.completion(
        model=model,
        messages=parsed_messages,
        **kwargs
    )

    return response.choices[0].message.content

print("Setup complete for gpt-5. Helper functions and code context are ready.")

Setup complete for gpt-5. Helper functions and code context are ready.


In [12]:
master_prompt = [
    {
        "role": "system",
        "content": dedent("""
        You are an expert prompt engineer, and your task is to create a high-quality,
        optimized system prompt based on a user's specification.

        ## Process:

        1. Start by asking me to provide the task I want you to generate a prompt for.
        2. Once I provide the task, ask me questions to clarify any doubts or missing information.
        3. Once you have the necessary information, create a detailed and effective system prompt that
            I can use with an AI system to tackle the task at hand.
        4. When I write "GENERATE", you will generate the final prompt based on the information
            we have discussed.

        ## Generated prompt output rules:
        * Output only the text of the generated prompt between <prompt>[GENERATED PROMPT]</prompt>
        * Do not output any triple backticks nor code blocks, just the contents of the generated prompt 
            between the XML tags.
        * Leverage markdown and prompt engineering best practices to structure the prompt.

        ## Examples of responses for generated prompts:
        <prompt>
        You are a knowledgeable and supportive AI DevOps tutor specializing in automation, observability, and AWS. Your role is to guide a beginner who is currently learning the basics of AWS and aims to achieve the AWS Certified Developer - Associate certification. 

        Provide practical, hands-on learning experiences and real-world project ideas that align with my learning goals. Break down complex topics into understandable segments and include the following: 
        
        1. **Foundational Concepts**: Explain essential DevOps principles and practices with a focus on automation and observability.
        2. **AWS Services**: Introduce relevant AWS services, their use cases, and how they integrate into DevOps workflows.
        3. **Real-World Projects**: Suggest projects that help apply theoretical knowledge in practical contexts, actively involving automation and observability.
        4. **Study Resources**: Share study materials, tools, and best practices to prepare for the AWS Certified Developer - Associate exam.
        5. **Interactive Exercises**: Create exercises that reinforce concepts and give me hands-on experience with AWS and DevOps practices.
        
        Encourage questions and provide answers constructively, promoting a clear understanding and a smooth learning journey towards certification.
        </prompt>
        """)
    }
]
print("Done!")

Done!


In [13]:
conversation_history = master_prompt.copy()
generated_prompt = None

print("Starting interactive prompt generation session...")
print("Type 'quit' to exit the conversation.\n")

initial_response = get_completion(conversation_history)
print("AI: ", initial_response)
print("-" * 50 + "\n")

while True:
    user_input = input("You: ").strip()

    if user_input.lower() in ["quit", "exit", "stop"]:
        print("Ending conversation.")
        break

    conversation_history.append({
        "role": "user",
        "content": user_input
    })

    ai_response = get_completion(conversation_history)

    conversation_history.append({
        "role": "assistant",
        "content": ai_response
    })

    print("AI: ", ai_response)
    print("-" * 50 + "\n")

    if "<prompt>" in ai_response:
        generated_prompt = ai_response.split("<prompt>")[1].rsplit("</prompt>")[0].strip()
        print("Final prompt has been generated and stored!")
        break

if generated_prompt:
    print("=" * 60 + "\n")
    print("Generated prompt:\n")
    print(generated_prompt)
else:
    print("No final prompt was generated in this session")

Starting interactive prompt generation session...
Type 'quit' to exit the conversation.

AI:  What task would you like me to generate a prompt for? Once you share it, I’ll ask a few clarifying questions to tailor the prompt precisely to your needs.
--------------------------------------------------



You:  I'm thinking about some template for prompting, that can be used and adapted to user needs. Like a tool box full with prompt templates: For ex: I've identified a bug and i open the bug-template that contains all that i need in order to specify and then obtain from AI. How to do that?


AI:  Great idea—a reusable prompt toolbox is very powerful. Per our process, let’s start by picking the first task you want a prompt for.

Which template would you like to create first?
- Example options: Bug report intake, Bug triage/reproduction, Feature request, Code review, Debugging assistant, Research brief, Meeting summary, PR description, Data analysis, Brainstorming.

If you want to start with a “Bug template,” please confirm and answer the questions below. If you prefer a different template, tell me which and I’ll tailor the questions.

General toolbox questions
1) Target platform(s): Which AI system(s) will use these prompts (e.g., ChatGPT, Claude, local LLM)? Any context window limits?
2) Template format: Do you want
   - System prompts for the AI’s behavior,
   - User-fillable templates with placeholders,
   - Or both? Any preference for Markdown vs JSON?
3) Reusability: Any variable/placeholder naming convention you prefer (e.g., {{APP_NAME}}, <<ENV>>) and a standard sect